In [ ]:
import cv2
import os
from ultralytics import solutions
import numpy as np

# Pastikan Anda mengganti path ini dengan lokasi file video yang benar.
VIDEO_PATH = "../examples/test-video/10s.mp4" 
OUTPUT_IMAGE_PATH = "final_aggregated_heatmap.png" # Path untuk gambar output

# Pastikan model yang Anda gunakan sudah diunduh (misalnya, yolo11n.pt)
MODEL_PATH = "../models/objectDetections/best.pt"

cap = cv2.VideoCapture(VIDEO_PATH)
assert cap.isOpened(), "Error reading video file"

# Inisialisasi heatmap object
heatmap = solutions.Heatmap(
    show=False, 
    model=MODEL_PATH, 
    colormap=cv2.COLORMAP_PARULA, 
    # region=region_points, 
    # classes=[0, 2], 
    verbose=False
)

# Variabel untuk menyimpan frame heatmap terakhir
last_heatmap_frame = None

# Process video
while cap.isOpened():
    success, im0 = cap.read()

    if not success:
        print("Video frame is empty atau processing is complete.")
        break

    # Lakukan pemrosesan heatmap.
    results = heatmap(im0)

    if results.plot_im is not None:
        last_heatmap_frame = results.plot_im.copy()
        print("Frame terakhir diperbarui...") 

    # --- BAGIAN INI DIHAPUS: video_writer.write tidak diperlukan ---
    # video_writer.write(results.plot_im)
    # --------------------------------------------------------

cap.release()
# --- video_writer.release() juga dihapus ---


# ==========================================================
## Penyimpanan Gambar Terakhir (Frame Heatmap Agregat)
# ==========================================================

if last_heatmap_frame is not None:
    # Simpan frame heatmap terakhir ke file gambar
    success = cv2.imwrite(OUTPUT_IMAGE_PATH, last_heatmap_frame)
    
    if success:
        print(f"\n✅ Berhasil! Heatmap agregat final disimpan sebagai: {OUTPUT_IMAGE_PATH}")
    else:
        print(f"\n❌ Gagal menyimpan gambar ke: {OUTPUT_IMAGE_PATH}")
else:
    print("\n⚠️ Tidak ada frame yang diproses. Pastikan path video sudah benar.")

cv2.destroyAllWindows()

Ultralytics Solutions:  {'source': None, 'model': '../models/objectDetections/best.pt', 'classes': None, 'show_conf': True, 'show_labels': True, 'region': None, 'colormap': 12, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'cropped-detections', 'json_file': None, 'line_width': 2, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': False, 'iou': 0.7, 'conf': 0.25, 'device': None, 'max_det': 300, 'half': False, 'tracker': 'botsort.yaml', 'verbose': False, 'data': 'images'}
Frame terakhir diperbarui...
Frame terakhir diperbarui...
Frame terakhir diperbarui...
Frame terakhir diperbarui...
Frame terakhir diperbarui...
Frame terakhir diperbarui...
Frame terakhir diperbarui...
Frame terakhir diperbarui...
Frame terakhir diperbarui...
Frame terakhir diperbarui...
Frame terakhir diperbarui...
Frame terak

## Modifikasi Gemini

In [1]:
import cv2
import os
import numpy as np
import supervision as sv
from ultralytics import YOLO
from sports.common.view import ViewTransformer
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from typing import Optional, Tuple

# --- Impor Konfigurasi dan Helper dari Direktori Anda ---
# Pastikan Anda menjalankan skrip ini dari direktori yang dapat mengakses 'config/'
from config.tennis_court import TennisCourtConfiguration
from config.tennis_annotator import draw_court

# --- PATH KONFIGURASI (Sesuai dengan struktur folder Anda) ---
# Gunakan path relatif dari root proyek jika skrip ini berada di 'scripts/'
VIDEO_PATH = "../examples/test-video/10s.mp4"
MODEL_PATH = "../models/objectDetections/best.pt" # Model deteksi objek (Pemain/Bola)
MODEL_FIELD_PATH = "../models/courtKeyPoints/best.pt" # Asumsi model keypoint ada di sini
OUTPUT_IMAGE_PATH = "output/heatmap/tennis_court_heatmap_players.png"

# Buat direktori output
os.makedirs("output/heatmap/", exist_ok=True)

# --- KONSTANTA ---
CONFIG = TennisCourtConfiguration() # Konfigurasi lapangan
PLAYER_ID = 1 # Asumsi ID kelas pemain adalah 1 (sesuai Untitled-1.py)
MIN_CONFIDENCE = 0.3
REQUIRED_KP_POINTS = 4 # Minimal keypoint yang dibutuhkan untuk homografi

# --- INISIALISASI MODEL DAN STRUKTUR DATA ---
model = YOLO(MODEL_PATH)
model_field = YOLO(MODEL_FIELD_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)
assert cap.isOpened(), "Error reading video file"

# List untuk menyimpan semua koordinat pemain yang sudah diproyeksikan (di lapangan 2D)
all_pitch_points = []
transformer: Optional[ViewTransformer] = None


# Fungsi untuk mendapatkan Transformer Homografi (Diambil dari Untitled-1.py)
def get_homography_transformer(frame: np.ndarray, field_model: YOLO, config: TennisCourtConfiguration) -> Optional[ViewTransformer]:
    """Mencoba menghitung transformer homografi dari satu frame."""
    try:
        result = field_model(frame, conf=MIN_CONFIDENCE)[0]
        key_points = sv.KeyPoints.from_ultralytics(result)
        filter_kp = key_points.confidence[0] > 0.5

        if np.sum(filter_kp) >= REQUIRED_KP_POINTS:
            frame_reference_points = key_points.xy[0][filter_kp]
            # Gunakan keypoints yang sudah diurutkan dari konfigurasi Anda
            pitch_reference_points = np.array(config.keypoints)[filter_kp]
            # Penting: source (video) ke target (pitch 2D) untuk tampilan radar
            transformer = ViewTransformer(source=frame_reference_points, target=pitch_reference_points)
            return transformer
    except Exception as e:
        # print(f"Homography calculation failed: {e}")
        pass
    return None

# --- LOOP PEMROSESAN VIDEO ---
print("Mulai memproses video dan mengumpulkan data pemain...")

frame_count = 0
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        break

    # 1. Coba hitung/perbarui Homografi (Hanya jika belum berhasil/di awal)
    if transformer is None:
        transformer = get_homography_transformer(im0, model_field, CONFIG)
        if transformer is None:
            # print("Homografi gagal. Melewati frame ini.")
            continue

    # 2. Deteksi Pemain
    results = model(im0, conf=MIN_CONFIDENCE)[0]
    detections = sv.Detections.from_ultralytics(results)
    
    # Filter hanya pemain (asumsi ID 1)
    players_detections = detections[detections.class_id == PLAYER_ID]

    if len(players_detections) > 0:
        # 3. Proyeksi Titik Pemain
        # Ambil titik tengah bawah (anchors)
        frame_players_xy = players_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
        pitch_players_xy = transformer.transform_points(points=frame_players_xy)
        
        # 4. Simpan Titik yang berhasil diproyeksikan
        # Hapus titik NaN/Inf yang mungkin muncul dari proyeksi yang buruk
        valid_points = pitch_players_xy[~np.isnan(pitch_players_xy).any(axis=1)]
        all_pitch_points.extend(valid_points)
        
    frame_count += 1
    if frame_count % 100 == 0:
        print(f"Processed {frame_count} frames. Collected {len(all_pitch_points)} player coordinates.")

cap.release()
print(f"\nProses pengumpulan data selesai. Total koordinat yang dikumpulkan: {len(all_pitch_points)}")

# ==========================================================
## Pembuatan Heatmap Kustom pada Denah Lapangan
# ==========================================================

if not all_pitch_points:
    print("\n⚠️ Tidak ada data pemain yang terkumpul atau homografi gagal di semua frame.")
    cv2.destroyAllWindows()
    exit()

# 1. Buat kanvas denah lapangan
RADAR_SCALE = 0.4
RADAR_PADDING = 50
heatmap_canvas = draw_court(CONFIG, scale=RADAR_SCALE, padding=RADAR_PADDING)
h_canvas, w_canvas, _ = heatmap_canvas.shape
pitch_points_np = np.array(all_pitch_points)

# 2. Persiapan Data untuk KDE
# Skalakan koordinat 2D lapangan agar sesuai dengan ukuran kanvas
scaled_points = (pitch_points_np * RADAR_SCALE) + RADAR_PADDING
# KDE bekerja paling baik jika hanya menggunakan titik yang memiliki sebaran data
# Kita hanya perlu koordinat x dan y yang diproyeksikan
X_scaled = scaled_points[:, 0]
Y_scaled = scaled_points[:, 1]
data = np.vstack([X_scaled, Y_scaled])

# Batasi kernel pada area yang masuk akal di kanvas
xmin, xmax = 0, w_canvas
ymin, ymax = 0, h_canvas

# 3. Hitung KDE (Estimasi Kepadatan Kernel)
try:
    # Bandwidth yang lebih kecil menghasilkan bintik yang lebih tajam
    kde = gaussian_kde(data, bw_method=0.1) 
    # Buat grid untuk mengevaluasi kepadatan
    xi, yi = np.mgrid[xmin:xmax:w_canvas*1j, ymin:ymax:h_canvas*1j]
    coords = np.vstack([xi.ravel(), yi.ravel()])
    
    # Hitung nilai kepadatan
    density = kde(coords).reshape(xi.shape)
    
    # Normalisasi densitas ke 0-255 dan ubah ke 8-bit
    density_normalized = (density - density.min()) / (density.max() - density.min())
    density_uint8 = (density_normalized * 255).astype(np.uint8)
    
    # Rotasi/Flip karena mgrid menghasilkan matriks terbalik (tergantung implementasi)
    heatmap_mask = density_uint8.T 
    
    # Terapkan Colormap (Misalnya PARULA)
    heatmap_colored = cv2.applyColorMap(heatmap_mask, cv2.COLORMAP_JET)

    # 4. Gabungkan Heatmap dengan Denah Lapangan (Overlay)
    # Jadikan area dengan kepadatan 0 (luar) transparan
    mask_3channel = np.stack([heatmap_mask] * 3, axis=-1)
    
    # Buat mask biner untuk overlay (area yang tidak nol)
    alpha_mask = (heatmap_mask > 0).astype(np.uint8) * 255
    alpha_mask_3ch = cv2.cvtColor(alpha_mask, cv2.COLOR_GRAY2BGR)

    # Overlay dengan transparansi (misalnya 50% transparansi)
    OVERLAY_OPACITY = 0.6
    
    # Gunakan mask untuk hanya menimpa di area yang memiliki data
    result = np.where(alpha_mask_3ch > 0, 
                      cv2.addWeighted(heatmap_colored, OVERLAY_OPACITY, heatmap_canvas, 1 - OVERLAY_OPACITY, 0),
                      heatmap_canvas)
    
    final_image = result.astype(np.uint8)

except Exception as e:
    print(f"\n❌ Gagal membuat Heatmap KDE: {e}. Menggunakan peta titik mentah.")
    # Jika gagal membuat KDE, setidaknya gambar semua titik mentah
    final_image = draw_court(CONFIG, scale=RADAR_SCALE, padding=RADAR_PADDING)
    # Konversi ke int agar draw_points_on_court bisa bekerja
    points_to_draw = scaled_points.astype(int) 
    for p in points_to_draw:
        cv2.circle(final_image, (p[0], p[1]), 3, sv.Color.RED.as_bgr(), -1)

# --- Penyimpanan Gambar Hasil Akhir ---
success = cv2.imwrite(OUTPUT_IMAGE_PATH, final_image)

if success:
    print(f"\n✅ Berhasil! Heatmap pemain pada denah lapangan disimpan sebagai: {OUTPUT_IMAGE_PATH}")
else:
    print(f"\n❌ Gagal menyimpan gambar ke: {OUTPUT_IMAGE_PATH}")

cv2.destroyAllWindows()

Mulai memproses video dan mengumpulkan data pemain...

0: 736x1280 1 tennis_court, 83.5ms
Speed: 7.8ms preprocess, 83.5ms inference, 184.3ms postprocess per image at shape (1, 3, 736, 1280)

0: 448x736 1 ball, 2 players, 39.9ms
Speed: 1.7ms preprocess, 39.9ms inference, 3.4ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 2 players, 38.0ms
Speed: 1.8ms preprocess, 38.0ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 2 players, 37.7ms
Speed: 1.3ms preprocess, 37.7ms inference, 1.8ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 2 players, 38.3ms
Speed: 1.2ms preprocess, 38.3ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 2 players, 38.1ms
Speed: 1.7ms preprocess, 38.1ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 3 players, 37.8ms
Speed: 1.3ms preprocess, 37.8ms inference, 1.7ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 2 players, 38.2ms
Speed: 1.2

In [3]:
import cv2
import os
import numpy as np
import supervision as sv
from ultralytics import YOLO
from sports.common.view import ViewTransformer
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from typing import Optional, Tuple

# --- Impor Konfigurasi dan Helper dari Direktori Anda ---
# Pastikan Anda menjalankan skrip ini dari direktori yang dapat mengakses 'config/'
from config.tennis_court import TennisCourtConfiguration
from config.tennis_annotator import draw_court

# --- PATH KONFIGURASI (Sesuai dengan struktur folder Anda) ---
# Gunakan path relatif dari root proyek jika skrip ini berada di 'scripts/'
VIDEO_PATH = "../examples/test-video/10s.mp4"
MODEL_PATH = "../models/objectDetections/best.pt" # Model deteksi objek (Pemain/Bola)
MODEL_FIELD_PATH = "../models/courtKeyPoints/best.pt" # Asumsi model keypoint ada di sini
OUTPUT_IMAGE_PATH = "output/heatmap/tennis_court_heatmap_players.png"

# Buat direktori output
os.makedirs("output/heatmap/", exist_ok=True)

# --- KONSTANTA ---
CONFIG = TennisCourtConfiguration() # Konfigurasi lapangan
PLAYER_ID = 0 # Asumsi ID kelas pemain adalah 1 (sesuai Untitled-1.py)
MIN_CONFIDENCE = 0.3
REQUIRED_KP_POINTS = 4 # Minimal keypoint yang dibutuhkan untuk homografi

# --- INISIALISASI MODEL DAN STRUKTUR DATA ---
model = YOLO(MODEL_PATH)
model_field = YOLO(MODEL_FIELD_PATH)
cap = cv2.VideoCapture(VIDEO_PATH)
assert cap.isOpened(), "Error reading video file"

# List untuk menyimpan semua koordinat pemain yang sudah diproyeksikan (di lapangan 2D)
all_pitch_points = []
transformer: Optional[ViewTransformer] = None


# Fungsi untuk mendapatkan Transformer Homografi (Diambil dari Untitled-1.py)
def get_homography_transformer(frame: np.ndarray, field_model: YOLO, config: TennisCourtConfiguration) -> Optional[ViewTransformer]:
    """Mencoba menghitung transformer homografi dari satu frame."""
    try:
        result = field_model(frame, conf=MIN_CONFIDENCE)[0]
        key_points = sv.KeyPoints.from_ultralytics(result)
        filter_kp = key_points.confidence[0] > 0.5

        if np.sum(filter_kp) >= REQUIRED_KP_POINTS:
            frame_reference_points = key_points.xy[0][filter_kp]
            # Gunakan keypoints yang sudah diurutkan dari konfigurasi Anda
            pitch_reference_points = np.array(config.keypoints)[filter_kp]
            # Penting: source (video) ke target (pitch 2D) untuk tampilan radar
            transformer = ViewTransformer(source=frame_reference_points, target=pitch_reference_points)
            return transformer
    except Exception as e:
        # print(f"Homography calculation failed: {e}")
        pass
    return None

# --- LOOP PEMROSESAN VIDEO ---
print("Mulai memproses video dan mengumpulkan data pemain...")

frame_count = 0
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        break

    # 1. Coba hitung/perbarui Homografi (Hanya jika belum berhasil/di awal)
    if transformer is None:
        transformer = get_homography_transformer(im0, model_field, CONFIG)
        if transformer is None:
            # print("Homografi gagal. Melewati frame ini.")
            continue

    # 2. Deteksi Pemain
    results = model(im0, conf=MIN_CONFIDENCE)[0]
    detections = sv.Detections.from_ultralytics(results)
    
    # Filter hanya pemain (asumsi ID 1)
    players_detections = detections[detections.class_id == PLAYER_ID]

    if len(players_detections) > 0:
        # 3. Proyeksi Titik Pemain
        # Ambil titik tengah bawah (anchors)
        frame_players_xy = players_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
        pitch_players_xy = transformer.transform_points(points=frame_players_xy)
        
        # 4. Simpan Titik yang berhasil diproyeksikan
        # Hapus titik NaN/Inf yang mungkin muncul dari proyeksi yang buruk
        valid_points = pitch_players_xy[~np.isnan(pitch_players_xy).any(axis=1)]
        all_pitch_points.extend(valid_points)
        
    frame_count += 1
    if frame_count % 100 == 0:
        print(f"Processed {frame_count} frames. Collected {len(all_pitch_points)} player coordinates.")

cap.release()
print(f"\nProses pengumpulan data selesai. Total koordinat yang dikumpulkan: {len(all_pitch_points)}")

# ==========================================================
## Pembuatan Heatmap Kustom pada Denah Lapangan
# ==========================================================

if not all_pitch_points:
    print("\n⚠️ Tidak ada data pemain yang terkumpul atau homografi gagal di semua frame.")
    cv2.destroyAllWindows()
    exit()

# 1. Buat kanvas denah lapangan
RADAR_SCALE = 0.4
RADAR_PADDING = 50
heatmap_canvas = draw_court(CONFIG, scale=RADAR_SCALE, padding=RADAR_PADDING)
h_canvas, w_canvas, _ = heatmap_canvas.shape
pitch_points_np = np.array(all_pitch_points)

# 2. Persiapan Data untuk KDE
# Skalakan koordinat 2D lapangan agar sesuai dengan ukuran kanvas
scaled_points = (pitch_points_np * RADAR_SCALE) + RADAR_PADDING
# KDE bekerja paling baik jika hanya menggunakan titik yang memiliki sebaran data
# Kita hanya perlu koordinat x dan y yang diproyeksikan
X_scaled = scaled_points[:, 0]
Y_scaled = scaled_points[:, 1]
data = np.vstack([X_scaled, Y_scaled])

# Batasi kernel pada area yang masuk akal di kanvas
xmin, xmax = 0, w_canvas
ymin, ymax = 0, h_canvas

# 3. Hitung KDE (Estimasi Kepadatan Kernel)
try:
    # Bandwidth yang lebih kecil menghasilkan bintik yang lebih tajam
    kde = gaussian_kde(data, bw_method=0.1) 
    # Buat grid untuk mengevaluasi kepadatan
    xi, yi = np.mgrid[xmin:xmax:w_canvas*1j, ymin:ymax:h_canvas*1j]
    coords = np.vstack([xi.ravel(), yi.ravel()])
    
    # Hitung nilai kepadatan
    density = kde(coords).reshape(xi.shape)
    
    # Normalisasi densitas ke 0-255 dan ubah ke 8-bit
    density_normalized = (density - density.min()) / (density.max() - density.min())
    density_uint8 = (density_normalized * 255).astype(np.uint8)
    
    # Rotasi/Flip karena mgrid menghasilkan matriks terbalik (tergantung implementasi)
    heatmap_mask = density_uint8.T 
    
    # Terapkan Colormap (Misalnya PARULA)
    heatmap_colored = cv2.applyColorMap(heatmap_mask, cv2.COLORMAP_JET)

    # 4. Gabungkan Heatmap dengan Denah Lapangan (Overlay)
    # Jadikan area dengan kepadatan 0 (luar) transparan
    mask_3channel = np.stack([heatmap_mask] * 3, axis=-1)
    
    # Buat mask biner untuk overlay (area yang tidak nol)
    alpha_mask = (heatmap_mask > 0).astype(np.uint8) * 255
    alpha_mask_3ch = cv2.cvtColor(alpha_mask, cv2.COLOR_GRAY2BGR)

    # Overlay dengan transparansi (misalnya 50% transparansi)
    OVERLAY_OPACITY = 0.6
    
    # Gunakan mask untuk hanya menimpa di area yang memiliki data
    result = np.where(alpha_mask_3ch > 0, 
                      cv2.addWeighted(heatmap_colored, OVERLAY_OPACITY, heatmap_canvas, 1 - OVERLAY_OPACITY, 0),
                      heatmap_canvas)
    
    final_image = result.astype(np.uint8)

except Exception as e:
    print(f"\n❌ Gagal membuat Heatmap KDE: {e}. Menggunakan peta titik mentah.")
    # Jika gagal membuat KDE, setidaknya gambar semua titik mentah
    final_image = draw_court(CONFIG, scale=RADAR_SCALE, padding=RADAR_PADDING)
    # Konversi ke int agar draw_points_on_court bisa bekerja
    points_to_draw = scaled_points.astype(int) 
    for p in points_to_draw:
        cv2.circle(final_image, (p[0], p[1]), 3, sv.Color.RED.as_bgr(), -1)

# --- Penyimpanan Gambar Hasil Akhir ---
success = cv2.imwrite(OUTPUT_IMAGE_PATH, final_image)

if success:
    print(f"\n✅ Berhasil! Heatmap pemain pada denah lapangan disimpan sebagai: {OUTPUT_IMAGE_PATH}")
else:
    print(f"\n❌ Gagal menyimpan gambar ke: {OUTPUT_IMAGE_PATH}")

cv2.destroyAllWindows()

Mulai memproses video dan mengumpulkan data pemain...

0: 736x1280 1 tennis_court, 85.1ms
Speed: 4.5ms preprocess, 85.1ms inference, 1.8ms postprocess per image at shape (1, 3, 736, 1280)

0: 448x736 1 ball, 2 players, 41.4ms
Speed: 1.6ms preprocess, 41.4ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 2 players, 37.6ms
Speed: 1.2ms preprocess, 37.6ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 2 players, 37.9ms
Speed: 1.2ms preprocess, 37.9ms inference, 1.7ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 2 players, 37.9ms
Speed: 1.4ms preprocess, 37.9ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 2 players, 38.2ms
Speed: 1.3ms preprocess, 38.2ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 3 players, 38.5ms
Speed: 1.3ms preprocess, 38.5ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 736)

0: 448x736 2 players, 38.5ms
Speed: 1.5ms